In [1]:
class RBNode:

    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        self.colour = "R"

    def get_uncle(self):
        return

    def is_leaf(self):
        return self.left == None and self.right == None

    def is_left_child(self):
        return self == self.parent.left

    def is_right_child(self):
        return not self.is_left_child()

    def is_red(self):
        return self.colour == "R"

    def is_black(self):
        return not self.is_red()

    def make_black(self):
        self.colour = "B"

    def make_red(self):
        self.colour = "R"

    def get_brother(self):
        if self.parent.right == self:
            return self.parent.left
        return self.parent.right

    def get_uncle(self):
        return self.parent.get_brother()

    def uncle_is_black(self):
        if self.get_uncle() == None:
            return True
        return self.get_uncle().is_black()

    def __str__(self):
        return "(" + str(self.value) + "," + self.colour + ")"

    def __repr__(self):
         return "(" + str(self.value) + "," + self.colour + ")"

    def rotate_right(self):
        #TODO
        return 0

    def rotate_left(self):
        #TODO
        return 0
                    
        
    

In [ ]:
class RBTree:

    def __init__(self,):
        self.root = None

    def is_empty(self,):
        return self.root == None

    def get_height(self,):
        if self.is_empty():
            return 0
        return self.__get_height(self.root)

    def __get_height(self, node):
        if node == None:
            return 0
        return 1 + max(self.__get_height(node.left), self.__get_height(node.right))

    def insert(self, value):
        if self.is_empty():
            self.root = RBNode(value)
            self.root.make_black()
        else:
            self.__insert(self.root, value)

    def __insert(self, node, value):
        if value < node.value:
            if node.left == None:
                node.left = RBNode(value)
                node.left.parent = node
                self.fix(node.left)
            else:
                self.__insert(node.left, value)
        else:
            if node.right == None:
                node.right = RBNode(value)
                node.right.parent = node
                self.fix(node.right)
            else:
                self.__insert(node.right, value)

    def fix(self, node):
        #You may alter code in this method if you wish, it's merely a guide.
        if node.parent == None:
            node.make_black()
        # while node != None and node.parent != None and node.parent.is_red(): 
            #TODO
        self.root.make_black()

    def __str__(self):
        if self.is_empty():
            return "[]"
        return "[" + self.__str_helper(self.root) + "]"

    def __str_helper(self, node):
        if node.is_leaf():
            return "[" + str(node) + "]"
        if node.left == None:
            return "[" + str(node) + " -> " + self.__str_helper(node.right) + "]"
        if node.right == None:
            return "[" +  self.__str_helper(node.left) + " <- " + str(node) + "]"
        return "[" + self.__str_helper(node.left) + " <- " + str(node) + " -> " + self.__str_helper(node.right) + "]"


In [1]:
import math
import random
import time
import timeit 
import matplotlib.pyplot as plt
import numpy as np

In [157]:
# Part 2.1

# k = log(n)
# k = log(1 000) = 10
# arrays = 1, 2, 4, 8, 16, 32, 64, 128, 256, 512  # lengths

class BSearch_Fast():
    def __init__(self, items_in: list):
        items_in.sort()
        items = list(items_in)
        self.n = len(items)
        # calculate k value and init lists
        k = math.ceil(math.log2(self.n+1))
        self.lists = [[] for _ in range(k)]

        # find the equivalent binary number
        n_in_binary = str(bin(self.n)[2:][::-1])
        idx = 0
        for i in range(k):
            if n_in_binary[i] == '1':
                for _ in range(2**i):
                    # add items to list if the digit is '1'
                    if (idx >= len(items)): break
                    self.lists[i].append(items[idx])
                    idx += 1

    def get_lists(self):
        return self.lists

    def search(self, to_find):
        # iterate over all lists and bsearch on them
        for item_list in self.lists:
            if (len(item_list) > 0):
                found = self.__bsearch_helper(item_list, to_find)
                if found: return True
        return False

    def __bsearch_helper(self, item_list, to_find):
        # straightforward binary search
        if len(item_list) == 0:
            return -1

        lower=0
        upper=len(item_list)-1
        while lower < upper:
            mid = (lower+upper)//2
            if item_list[mid] == to_find:
                return mid
            if item_list[mid] < to_find:
                lower = mid+1
            else:
                upper=mid
        if item_list[lower]==to_find:
            return lower
        return -1
 
    def insert(self, item):
        self.lists[0].append(item)
        if len(self.lists[0]) == 1: # no need to modify lists
            return True
        
        # initialize combine_lists
        combine_lists = self.lists[0]

        # while searching for empty list, add current list
        idx = 1
        while len(self.lists[idx]) > 0:
            combine_lists += list(self.lists[idx])
            idx += 1
            if (idx == len(self.lists)): break
        
        # we need another list
        if idx == len(self.lists):
            self.lists.append([])
        
        # combine the previous lists and sort
        self.lists[idx] = list(combine_lists)
        self.lists[idx].sort()
        # clear out all previous lists
        for i in range(idx):
            self.lists[i] = []

        return item

    def delete(self, item):
        # find the idx of the lowest non-empty list
        idx = 0
        while len(self.lists[idx]) == 0:
            idx += 1
            if (idx == len(self.lists)): break

        found = False
        search_idx = idx
        # find item and delete
        while not found:
            search = self.__bsearch_helper(self.lists[search_idx], item)

            # item found, now delete
            if (search != -1):
                found = True
                self.lists[search_idx].pop(search)

                # must take an item from the smallest non-empty list
                # and place into the list with the deleted item
                if (search_idx != idx):
                    fix_hole = self.lists[idx].pop()
                    self.lists[search_idx].append(fix_hole)
                    self.lists[search_idx].sort()
                break
            else:
                search_idx += 1
                if search_idx == len(self.lists): break

        if found and search_idx != 0:
            # move the disrupted list into all the smaller, empty lists
            if (idx != 0):
                self.lists[0].append(self.lists[idx][0])
            for i in range(1, idx):
                self.lists[i] = self.lists[idx][2**(i)-1:(2**(i+1))-1]
            # clear the first non-empty list
            if (idx != 0):
                self.lists[idx] = []
                if len(self.lists[-1]) == 0:
                    self.lists.pop()
        
        return found

test = BSearch_Fast([i for i in range(16)])
test.delete(0)
print(test.get_lists())

[[1], [2, 3], [4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]


In [ ]:
def binary_search_1(item_list, to_find):
    lower=0
    upper=len(item_list)-1
    while lower < upper:
        mid = (lower+upper)//2
        if item_list[mid] == to_find:
            return True
        if item_list[mid] < to_find:
            lower = mid+1
        else:
            upper=mid
    return item_list[lower]==to_find

def binary_search_2(item_list, to_find):
    lower=0
    upper=len(item_list)-1
    while lower <= upper:
        mid = (lower+upper)//2
        if item_list[mid] == to_find:
            return True
        if item_list[mid] < to_find:
            lower = mid+1
        else:
            upper=mid-1
    return item_list[lower]==to_find

def binary_search_3(item_list, to_find):
    left=0
    right=len(item_list)-1
    while left != right:
        mid = (left+right)//2
        if item_list[mid] < to_find:
            left = mid+1
        elif item_list[mid] > to_find:
            right = mid
        else:
            return True
    return item_list[left]==to_find

def binary_search_4(item_list, to_find, left, right):
    if left == right:
        return item_list[left] == to_find
    mid = (left+right)//2
    if item_list[mid] < to_find:
        return binary_search_4(item_list, to_find, mid+1, right)
    return binary_search_4(item_list, to_find, left, mid)

def create_custom_list(length, max_value, item=None, item_index=None):
    random_list = [random.randint(0,max_value) for i in range(length)]
    if item!= None:
        random_list.insert(item_index,item)
    return random_list

In [ ]:
sorted_lists = []

for i in range(50):
    sorted_lists.append(create_custom_list(5000, 20000).sort())


def bs_1_speed(list, num):
    start = timeit.default_timer()
    binary_search_1(list, num)
    end = timeit.default_timer()
    return end-start
def bs_2_speed(list, num):
    start = timeit.default_timer()
    binary_search_2(list, num)
    end = timeit.default_timer()
    return end-start
def bs_3_speed(list, num):
    start = timeit.default_timer()
    binary_search_3(list, num)
    end = timeit.default_timer()
    return end-start
def bs_4_speed(list, num):
    start = timeit.default_timer()
    binary_search_4(list, num)
    end = timeit.default_timer()
    return end-start
def bs_5_speed(list, num):
    bsearch = BSearch_Fast(list)
    start = timeit.default_timer()
    bsearch.search(num)
    end = timeit.default_timer()
    return end-start


odd_speed = []
even_speed = []

odd_speed.append(bs_1_speed(odd_list, 1))
odd_speed.append(bs_1_speed(odd_list, 1250))
odd_speed.append(bs_1_speed(odd_list, 2500))
odd_speed.append(bs_2_speed(odd_list, 1))
odd_speed.append(bs_2_speed(odd_list, 1250))
odd_speed.append(bs_2_speed(odd_list, 2500))
odd_speed.append(bs_3_speed(odd_list, 1))
odd_speed.append(bs_3_speed(odd_list, 1250))
odd_speed.append(bs_3_speed(odd_list, 2500))

even_speed.append(bs_1_speed(even_list, 0))
even_speed.append(bs_1_speed(even_list, 1249))
even_speed.append(bs_1_speed(even_list, 2499))
even_speed.append(bs_2_speed(even_list, 0))
even_speed.append(bs_2_speed(even_list, 1249))
even_speed.append(bs_2_speed(even_list, 2499))
even_speed.append(bs_3_speed(even_list, 0))
even_speed.append(bs_3_speed(even_list, 1249))
even_speed.append(bs_3_speed(even_list, 2499))

def draw_binary_search_speeds(run_arr, x_axis):
    x = np.arange(0, len(run_arr), 1)
    plt.figure(figsize=(20, 8))
    half_len = len(run_arr) // 2
    plt.bar(x[:half_len], run_arr[:half_len], color='orange')
    plt.bar(x[half_len:], run_arr[half_len:], color='blue')
    plt.axhline(np.mean(run_arr), color="red", linestyle="--", label="Avg")
    plt.xticks(x, x_axis)
    plt.xlabel("Sorting Algorithm and list type")
    plt.ylabel("Run time in ms in order of 1e-6")
    plt.title("Run time for Binary Search on Odd length list versus Even length list")
    plt.legend()
    plt.show()

bs_xaxis = ['BS1 Low', 'BS1 Mid', 'BS1 High', 'BS2 Low', 'BS2 Mid', 'BS2 High', 'BS3 Low', 'BS3 Mid', 'BS3 High']
bs_xaxis += ['BS1 Low', 'BS1 Mid', 'BS1 High', 'BS2 Low', 'BS2 Mid', 'BS2 High', 'BS3 Low', 'BS3 Mid', 'BS3 High']
draw_binary_search_speeds(odd_speed + even_speed, bs_xaxis)